# Population project
## Steven Turner

#### Imports and Definitions

In [1]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def exp(t, A, r):
    return A*np.exp(t*r)

def log(t, L, A, r):
    return L/(1+A*np.exp(-r*t))

def gom(t, L, b,r):
    return L*np.exp(-b*np.exp(-r*t))

def R2value(ydata, yapp):
    ymean = np.mean(ydata)
    R2 = 1.0 - (np.sum((yapp - ydata)**2))/(np.sum((ydata - ymean)**2))
    return R2

# Munich, Germany

#### Data

In [2]:
CensusFile = open('MunichPop.csv', 'r') #Don't you dare overwrite this file!
M = 50 #50 data points.
a = 0
b = 0
Gyears = np.zeros([M], dtype=int)
Gpop = np.zeros([M], dtype=int)
for line in CensusFile:
    if a == 0 or a==1:
        a=a+1
    else:
        values = line.split(',')
        Gyears[b] = int(values[0])
        Gpop[b] = int(values[1])
        b = b+1
        a = a+1
CensusFile.close()
GTime = Gyears - 1830 # base at 0
GPop_mil = Gpop/1.e6

#Floating point types, used by curve_fit
GTim_Float = GTime.astype(np.float64)
GPop_Float = Gpop.astype(np.float64)/1e6

#### Model

In [3]:
L = 1.5

### Exponetial model
EM_yval = np.log(GPop_mil)
[EM_r, EM_lnA] = np.polyfit(GTime,EM_yval, 1) #first value is r
EM_param, EM_cov = curve_fit(exp, GTim_Float, GPop_Float, p0=(np.exp(EM_lnA), EM_r)) #Nonlinear fit the first thing 
EM_model = exp(GTime, EM_param[0], EM_param[1])


### Logistic Model
LM_yval = np.log(L/GPop_mil - 1.0)
[LM_mr,LM_lnA] = np.polyfit(GTime, LM_yval,1)
LM_param, LM_cov = curve_fit(log, GTim_Float, GPop_Float, p0 = (L, np.exp(LM_lnA), -LM_mr)) #Nonlinear fit
LM_model = log(GTime, LM_param[0], LM_param[1], LM_param[2])

### Gompertz Model
GM_yval = np.log(np.log(L/GPop_mil))
[GM_slope, GM_intercept] = np.polyfit(GTime, GM_yval,1) #Linear least squares fit slope is -r and int is ln(b)
GM_param, GM_cov = curve_fit(gom, GTim_Float, GPop_Float, p0 = (L, np.exp(GM_intercept), -GM_slope)) #Nonlinear fit
GM_model = gom(GTime, GM_param[0], GM_param[1], GM_param[2])

#### Last one out Method

In [4]:
L = 1.5

### Exponetial model
EMP_yval = np.log(GPop_mil[0:M-1])
[EMP_r, EMP_lnA] = np.polyfit(GTime[0:M-1], EMP_yval, 1) #first value is r
EMP_param, EMP_cov = curve_fit(exp, GTim_Float[0:M-1], GPop_Float[0:M-1], p0=(np.exp(EMP_lnA), EMP_r)) #Nonlinear fit the first thing 
EMP_model = exp(GTime, EMP_param[0], EMP_param[1])

### Logistic Model
LMP_yval = np.log(L/GPop_mil[0:M-1] - 1.0)
[LMP_mr,LMP_lnA] = np.polyfit(GTime[0:M-1], LMP_yval,1)
LMP_param, LMP_cov = curve_fit(log, GTim_Float[0:M-1], GPop_Float[0:M-1], p0 = (L, np.exp(LMP_lnA), -LMP_mr)) #Nonlinear fit
LMP_model = log(GTime, LMP_param[0], LMP_param[1], LMP_param[2])

### Gompertz Model
GMP_yval = np.log(np.log(L/GPop_mil[0:M-1]))
[GMP_slope, GMP_intercept] = np.polyfit(GTime[0:M-1], GMP_yval ,1) #Linear least squares fit slope is -r and int is ln(b)
GMP_param, GMP_cov = curve_fit(gom, GTim_Float[0:M-1], GPop_Float[0:M-1], p0 = (L, np.exp(GMP_intercept), -GMP_slope)) #Nonlinear fit
GMP_model = gom(GTime, GMP_param[0], GMP_param[1], GMP_param[2])

#### Outputs

In [5]:
print "Exponetial Model"
print "--param A and r", EM_param
print "--R2 value", R2value(GPop_Float, EM_model)
print "--Final data point =", GPop_mil[M-1], "Estimated value = ", EM_model[M-1]
print
print "Exponetial Model prediction"
print "--param A and r", EMP_param
print "--R2 value", R2value(GPop_Float, EMP_model)
print "--Final data point in graph =", GPop_mil[M-1], "Estimated value = ", EMP_model[M-1]
print '--Error', ((EMP_model[M-1]-GPop_mil[M-1])/GPop_mil[M-1])*100
print
print "Logistic Model"
print "--Nonlinear L, A and r are ", LM_param
print "--R2 value", R2value(GPop_Float,LM_model)
print "--Final data point =", GPop_mil[M-1], "Estimated value = ", LM_model[M-1]
print
print "Logistic Model prediction"
print "--Nonlinear L, A and r are ", LMP_param
print "--R2 value", R2value(GPop_Float, LMP_model)
print "--Final data point =", GPop_mil[M-1], "Estimated value = ", LMP_model[M-1]
print '--Error', ((LMP_model[M-1]-GPop_mil[M-1])/GPop_mil[M-1])*100
print
print "Gompertz Model"
print "--Nonlinear L, A and r are ", GM_param
print "--R2 value", R2value(GPop_Float,GM_model)
print "--Final data point =", GPop_mil[M-1], "Estimated value = ", GM_model[M-1]
print
print "Gompertz Model prediction"
print "--Nonlinear L, A and r are ", GMP_param
print "--R2 value", R2value(GPop_Float, GMP_model)
print "--Final data point =", GPop_mil[M-1], "Estimated value = ", GMP_model[M-1]
print '--Error', ((GMP_model[M-1]-GPop_mil[M-1])/GPop_mil[M-1])*100

Exponetial Model
--param A and r [ 0.19485463  0.01112957]
--R2 value 0.938304130824
--Final data point = 1.305525 Estimated value =  1.39714619365

Exponetial Model prediction
--param A and r [ 0.19355238  0.01119634]
--R2 value 0.93824010918
--Final data point in graph = 1.305525 Estimated value =  1.40430813011
--Error 7.5665445023

Logistic Model
--Nonlinear L, A and r are  [  1.36421942  21.86682604   0.0321998 ]
--R2 value 0.990957131741
--Final data point = 1.305525 Estimated value =  1.27115542023

Logistic Model prediction
--Nonlinear L, A and r are  [  1.35955416  21.96109883   0.03234017]
--R2 value 0.990948489184
--Final data point = 1.305525 Estimated value =  1.2685681224
--Error -2.83080581362

Gompertz Model
--Nonlinear L, A and r are  [ 1.61705118  4.09568923  0.01637353]
--R2 value 0.989852032048
--Final data point = 1.305525 Estimated value =  1.29023037111

Gompertz Model prediction
--Nonlinear L, A and r are  [ 1.61259237  4.10074269  0.01642624]
--R2 value 0.98985

In [20]:
### German Pop
plt.figure(figsize=(6,8))
plt.plot(Gyears, GPop_mil, 'o', color='r', label='Munich Population')
plt.plot(Gyears, EM_model, color='b', label='Exponential Model') 
plt.plot(Gyears, LM_model, color='g', label='Logistic Model') 
plt.plot(Gyears, GM_model, color='orange', label='Gom Model')
plt.ylabel('Population of Munich (millions)')
plt.xlabel('Year')
plt.xlim([1825, 2010])
plt.legend(loc=2)
plt.title('Population of Munich and Best Fit Models')
plt.savefig('GPop_Models')
# plt.show()
plt.close()

### German Pop Predictions
plt.figure(figsize=(6,8))
plt.plot(Gyears, GPop_mil, 'o', color='r', label='Munich Population')
plt.plot(Gyears, EMP_model, color='b', label='exponential model') 
plt.plot(Gyears, LMP_model, color='g', label='logistic model') 
plt.plot(Gyears, GMP_model, color='orange', label='Gompertz model')
plt.ylabel('Population of Munich (millions)', fontsize='15')
plt.xlabel('Year', fontsize='15')
plt.xlim([1825, 2010])
plt.legend(loc=2)
plt.title('Munich', fontsize='20')
plt.savefig('GPop_Prediction')
# plt.show()
plt.close()

# US

#### US Data

In [7]:
CensusFile = open('USPopulation.csv', 'r') #Don't you dare overwrite this file!
N = 23 #23 data points.
i = 0
j = 0
USyears = np.zeros([N], dtype=int)
USPop = np.zeros([N], dtype=int)
for line in CensusFile:
    if i == 0 or i==1:
        i=i+1
    else:
        values = line.split(',')
        USyears[j] = int(values[0])
        USPop[j] = int(values[1])
        j = j+1
        i = i+1

CensusFile.close()
USTime = USyears - 1790 # base at 0
USPop_mil = USPop/1.e6

#Floating point types, used by curve_fit
USTime_fl = USTime.astype(np.float64)
USPop_fl = USPop.astype(np.float64)/1e6

#### US Model

In [8]:
LUSPop_mil = np.log(USPop_mil)
[re, lnAe] = np.polyfit(USTime,LUSPop_mil, 1) # Linear least squares fit
popt, pcov = curve_fit(exp, USTime_fl, USPop_fl, p0=(np.exp(lnAe),re)) #Nonlinear fit
USPopExpFinal = exp(USTime, popt[0], popt[1])

print "Linear A and r are", np.exp(lnAe), re
print "Nonlinear A and r are ", popt
print "Covariance matrix is ", pcov
print "Final data point =", USPop_mil[N-1], "Estimated value = ", USPopExpFinal[N-1]
print
print

Lguess = 350
yval = np.log(Lguess/USPop_mil - 1.0)
[mrl, lnAl] = np.polyfit(USTime, yval,1) #Linear least squares fit
popt, pcov = curve_fit(log, USTime_fl, USPop_fl, p0 = (Lguess, np.exp(lnAl), -mrl)) #Nonlinear fit
USPopLogFinal = log(USTime, popt[0], popt[1], popt[2])

print "Linear L, A and r are", Lguess, np.exp(lnAl), -mrl
print "Nonlinear L, A and r are ", popt
print "Covariance matrix is ", pcov
print "Final data point =", USPop_mil[N-1], "Estimated value = ", USPopLogFinal[N-1]
print
print

yval = np.log(np.log(Lguess/USPop_mil))
[mrl, lnAl] = np.polyfit(USTime, yval,1)
popt, pcov = curve_fit(gom, USTime_fl, USPop_fl, p0 = (Lguess, np.exp(lnAl), -mrl))
USPopGomFinal = gom(USTime, popt[0], popt[1], popt[2])

print "Linear L, A and r are", Lguess, np.exp(lnAl), -mrl
print "Nonlinear L, A and r are ", popt
print "Covariance matrix is ", pcov
print "Final data point =", USPop_mil[N-1], "Estimated value = ", USPopGomFinal[N-1]
print
print

plt.figure(figsize=(6,8))
plt.plot(USyears, USPop_mil, 'o', color='r', label='Munich Population')
plt.plot(USyears, USPopExpFinal, color='b', label='Exponential Model')
plt.plot(USyears, USPopLogFinal, color='g', label='Logistic Model')
plt.plot(USyears, USPopGomFinal, color='orange', label="Gompertz Model")
plt.ylabel('US Population (millions)', fontsize='10')
plt.xlabel('Year')
plt.xlim([1790, 2010])
plt.legend(loc=2)
plt.title('Population of US and Best Fit Models')
plt.savefig("USpop_model")
# plt.show()
plt.close()

Linear A and r are 6.31870740867 0.0196230740592
Nonlinear A and r are  [  1.63219275e+01   1.36356507e-02]
Covariance matrix is  [[  2.40886171e+00  -7.56436294e-04]
 [ -7.56436294e-04   2.45797872e-07]]
Final data point = 308.745538 Estimated value =  327.783264889


Linear L, A and r are 350 77.2862419809 0.026926590088
Nonlinear L, A and r are  [  4.86475249e+02   5.80898166e+01   2.07764405e-02]
Covariance matrix is  [[  1.22959404e+03  -2.03511070e+01  -2.82128109e-02]
 [ -2.03511070e+01   1.32328720e+01   1.70773237e-03]
 [ -2.82128109e-02   1.70773237e-03   7.72815131e-07]]
Final data point = 308.745538 Estimated value =  303.819041725


Linear L, A and r are 350 6.07845636783 0.014055025293
Nonlinear L, A and r are  [  1.38952460e+03   5.75990653e+00   6.08489989e-03]
Covariance matrix is  [[  4.19174056e+04   6.43867712e+00  -8.27268570e-02]
 [  6.43867712e+00   4.68641737e-03  -9.17449955e-06]
 [ -8.27268570e-02  -9.17449955e-06   1.66832554e-07]]
Final data point = 308.7455

#### US Prediction

In [21]:
LUSPop_mil = np.log(USPop_mil[0:N-1])
[re, lnAe] = np.polyfit(USTime[0:N-1],LUSPop_mil, 1) # Linear least squares fit
popt, pcov = curve_fit(exp, USTime_fl[0:N-1], USPop_fl[0:N-1], p0=(np.exp(lnAe),re)) #Nonlinear fit
USPopExpFinal = exp(USTime, popt[0], popt[1])

print "Linear A and r are", np.exp(lnAe), re
print "Nonlinear A and r are ", popt
print "Final data point =", USPop_mil[N-1], "Estimated value = ", USPopExpFinal[N-1]
print "R2 ", R2value(USPop_fl, USPopExpFinal)
print "Error ", ((USPopExpFinal[N-1]- USPop_mil[N-1])/USPop_mil[N-1])*100
print
print

Lguess = 350
yval = np.log(Lguess/USPop_mil[0:N-1] - 1.0)
[mrl, lnAl] = np.polyfit(USTime[0:N-1], yval,1) #Linear least squares fit
popt, pcov = curve_fit(log, USTime_fl[0:N-1], USPop_fl[0:N-1], p0 = (Lguess, np.exp(lnAl), -mrl)) #Nonlinear fit
USPopLogFinal = log(USTime, popt[0], popt[1], popt[2])

print "Linear L, A and r are", Lguess, np.exp(lnAl), -mrl
print "Nonlinear L, A and r are ", popt
print "Final data point =", USPop_mil[N-1], "Estimated value = ", USPopLogFinal[N-1]
print "R2 ", R2value(USPop_fl, USPopLogFinal)
print "Error ", ((USPopLogFinal[N-1]- USPop_mil[N-1])/USPop_mil[N-1])*100
print
print

yval = np.log(np.log(Lguess/USPop_mil[0:N-1]))
[mrl, lnAl] = np.polyfit(USTime[0:N-1], yval,1)
popt, pcov = curve_fit(gom, USTime_fl[0:N-1], USPop_fl[0:N-1], p0 = (Lguess, np.exp(lnAl), -mrl))
USPopGomFinal = gom(USTime, popt[0], popt[1], popt[2])

print "Linear L, A and r are", Lguess, np.exp(lnAl), -mrl
print "Nonlinear L, A and r are ", popt
print "Final data point =", USPop_mil[N-1], "Estimated value = ", USPopGomFinal[N-1]
print "R2 ", R2value(USPop_fl, USPopGomFinal)
print "Error ", ((USPopGomFinal[N-1]- USPop_mil[N-1])/USPop_mil[N-1])*100
print
print

plt.figure(figsize=(6,8))
plt.plot(USyears, USPop_mil, 'o', color='r', label='US Population')
plt.plot(USyears, USPopExpFinal, color='b', label='exponential model')
plt.plot(USyears, USPopLogFinal, color='g', label='logistic model')
plt.plot(USyears, USPopGomFinal, color='orange', label="Gompertz model")
plt.ylabel('US Population (millions)', fontsize='15')
plt.xlabel('Year', fontsize='15')
plt.xlim([1790, 2010])
plt.legend(loc=2)
plt.title('US', fontsize='20')
plt.savefig("USpop_prediction")
# plt.show()
plt.close()

Linear A and r are 6.07753319244 0.020179013696
Nonlinear A and r are  [  1.50530646e+01   1.41889945e-02]
Final data point = 308.745538 Estimated value =  341.436043916
R2  0.983711356076
Error  10.5881711288


Linear L, A and r are 350 74.2799468884 0.0263598052858
Nonlinear L, A and r are  [  4.44189039e+02   5.64257346e+01   2.15216199e-02]
Final data point = 308.745538 Estimated value =  296.981952879
R2  0.99712814801
Error  -3.81012311859


Linear L, A and r are 350 5.65785434413 0.0130306531833
Nonlinear L, A and r are  [  1.27468707e+03   5.71833565e+00   6.29719971e-03]
Final data point = 308.745538 Estimated value =  304.773829768
R2  0.998986942091
Error  -1.28640182397


